# Week 7 — Sheaves via Covers and Gluing (Interference)
We simulate two causal mechanisms that are only visible under different local conditions.
We define overlapping covers and check which edges look stable in each cover.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rng = np.random.RandomState(123)
T = 10000
theta = rng.uniform(0, 360, size=T)
mix = rng.normal(0, 1, size=T)
Z1 = rng.binomial(1, 0.5, size=T)
Z2 = rng.binomial(1, 0.5, size=T)

def bump(angle_deg, center, width=40):
    d = np.abs((angle_deg - center + 180) % 360 - 180)
    x = np.clip(1 - d/width, 0, 1)
    return 0.5 * (1 + np.cos(np.pi * (1 - x))) * (x > 0)

w1 = bump(theta, center=270, width=50)
w2 = bump(theta, center=90,  width=50)
E1 = w1 * Z1 + 0.1 * rng.normal(size=T)
E2 = w2 * Z2 + 0.1 * rng.normal(size=T)
Y  = 1.0 * E1 + 0.5 * E2 + rng.normal(scale=0.5, size=T)

df = pd.DataFrame({'theta':theta, 'mix':mix, 'E1':E1, 'E2':E2, 'Y':Y})

covers = {
    'WS': ((df['theta'] >= 250) & (df['theta'] <= 290)),
    'WL': ((df['theta'] >= 230) & (df['theta'] <= 310)),
    'E':  ((df['theta'] >= 70)  & (df['theta'] <= 110)),
    'LM': (df['mix'] < -0.5),
}
covers['WL∩LM'] = covers['WL'] & covers['LM']
covers['E∩LM']  = covers['E']  & covers['LM']

def standardize(X):
    m = X.mean(0); s = X.std(0); s[s==0] = 1.0
    return (X - m) / s

def edge_presence(X, y, thresh=0.2):
    Xs = standardize(X)
    ys = (y - y.mean()) / (y.std() if y.std() != 0 else 1.0)
    beta, *_ = np.linalg.lstsq(Xs, ys, rcond=None)
    e1 = float(abs(beta[0]) >= thresh)
    e2 = float(abs(beta[1]) >= thresh)
    return e1, e2, beta

def frequency_on_cover(mask, K=8, seed=123):
    idx = np.where(mask.values)[0]
    if len(idx) < 200:
        return np.nan, np.nan, 0
    rng = np.random.RandomState(seed)
    rng.shuffle(idx)
    shards = np.array_split(idx, K)
    pres = []
    for shard in shards:
        X = df.loc[shard, ['E1','E2']].values
        y = df.loc[shard, 'Y'].values
        e1, e2, _ = edge_presence(X, y)
        pres.append((e1, e2))
    pres = np.array(pres, dtype=float)
    return pres[:,0].mean(), pres[:,1].mean(), len(idx)

rows = []
for name, m in covers.items():
    f1, f2, n = frequency_on_cover(m)
    rows.append({'cover':name, 'N':n, 'freq_E1_to_Y':f1, 'freq_E2_to_Y':f2})

tab = pd.DataFrame(rows).sort_values('cover')
tab


In [ ]:
plt.figure(figsize=(6,4))
x = np.arange(len(tab))
plt.bar(x - 0.15, tab['freq_E1_to_Y'], width=0.3, label='E1→Y')
plt.bar(x + 0.15, tab['freq_E2_to_Y'], width=0.3, label='E2→Y')
plt.xticks(x, tab['cover'], rotation=45)
plt.ylim(0, 1.05)
plt.ylabel('Edge stability frequency')
plt.title('Stability varies by cover (local views)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()
